In [25]:
import pandas as pd
import re
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import defaultdict
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding,SimpleRNN, LSTM, Activation, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Flatten
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from keras.models import load_model

In [26]:
df= pd.read_csv('train_preprocess.tsv.txt',sep='\t', header=None)


In [27]:
df.head()

,0,1
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative


In [28]:
#Rename the column
df.rename(columns = {0:'text', 1:'labels' }, inplace = True)

In [29]:
df.shape

(11000, 2)

In [30]:
df['labels'].value_counts()

positive    6416
negative    3436
neutral     1148
Name: labels, dtype: int64

In [31]:
import sqlite3

db = sqlite3.connect('challenge.db')
mycursor = db.cursor()
query = "CREATE TABLE IF NOT EXISTS Table_1 (text varchar(255), label varchar(255));"
mycursor.execute(query)
db.commit()

In [32]:
#Cleansing
import re


def lowercase(text):
    return text.lower()

def remove_unnecessary_char(text):
    text = re.sub('\n','', text)
    text = re.sub('rt','', text)
    text = re.sub('user','', text)
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text)
    text = re.sub('  +',' ', text)
    return text

def remove_nonaplhanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text)
    text = re.sub('  +',' ', text) 
    return text


# Untuk Proses Cleaning Data
def preprocess(text):
    text = lowercase(text) # 1
    text = remove_unnecessary_char(text) # 2
    text = remove_nonaplhanumeric(text) # 3
    return text


def process_text(input_text):
    try: 
        output_text = preprocess(input_text)
        return output_text
    except:
        print("Text is unreadable")

In [33]:
df['text_clean'] = df.text.apply(process_text)

In [34]:
# Kita simpan teks ke dalam sebuah variabel
data_preprocessed = df.text_clean.tolist()

In [35]:
# to do Extraction feature, we use library "Sklearn or scikit-learn".
# Sklearn is library for Machine Learning task.
# "CountVectorizer" one of modul to do "BoW"
from sklearn.feature_extraction.text import CountVectorizer

# proced Feature Extraction
count_vect = CountVectorizer()
count_vect.fit(data_preprocessed)

X = count_vect.transform(data_preprocessed)
print ("Feature Extraction finish")

Feature Extraction finish


In [12]:
print(X)

  (0, 983)	1
  (0, 1095)	1
  (0, 1269)	1
  (0, 1687)	1
  (0, 1923)	1
  (0, 2308)	1
  (0, 3297)	1
  (0, 3415)	1
  (0, 3527)	1
  (0, 3588)	1
  (0, 4111)	1
  (0, 4168)	2
  (0, 4483)	1
  (0, 5899)	1
  (0, 6333)	1
  (0, 6375)	1
  (0, 6555)	1
  (0, 6617)	1
  (0, 6932)	1
  (0, 7138)	1
  (0, 8071)	1
  (0, 8228)	1
  (0, 8479)	1
  (0, 9316)	1
  (0, 9391)	1
  :	:
  (10999, 8666)	1
  (10999, 8831)	1
  (10999, 8833)	2
  (10999, 9101)	1
  (10999, 10342)	1
  (10999, 11105)	3
  (10999, 11701)	1
  (10999, 12534)	1
  (10999, 12535)	1
  (10999, 13383)	1
  (10999, 13420)	1
  (10999, 13632)	1
  (10999, 13678)	1
  (10999, 13701)	2
  (10999, 13837)	1
  (10999, 14161)	1
  (10999, 14227)	1
  (10999, 14374)	1
  (10999, 15449)	1
  (10999, 15627)	1
  (10999, 16096)	1
  (10999, 16132)	1
  (10999, 16303)	1
  (10999, 17102)	3
  (10999, 17134)	1


In [36]:
pickle.dump(count_vect, open("feature(countvetorizer_nn).pickle", "wb"))

In [41]:
from sklearn.model_selection import train_test_split

classes = df['labels']
classes

0        positive
1         neutral
2        positive
3        positive
4        negative
           ...   
10995    positive
10996    positive
10997     neutral
10998    negative
10999    positive
Name: labels, Length: 11000, dtype: object

In [45]:
y = pd.get_dummies(classes).values


In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size = 0.2)

In [47]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier() 
model.fit(X_train, y_train)

print ("Training selesai")

Training selesai


In [48]:
pickle.dump(model, open("model(countvetorizer_nn).pickle", "wb"))

In [49]:
#Evaluation
from sklearn.metrics import classification_report

test = model.predict(X_test)

print ("Testing selesai")

print(classification_report(y_test, test)) 

Testing selesai
              precision    recall  f1-score   support

    negative       0.80      0.78      0.79       703
     neutral       0.78      0.68      0.72       240
    positive       0.88      0.91      0.89      1257

    accuracy                           0.84      2200
   macro avg       0.82      0.79      0.80      2200
weighted avg       0.84      0.84      0.84      2200



In [50]:
# Untuk lebih menyakinkan lagi, kita juga bisa melakukan "Cross Validation"
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

kf = KFold(n_splits=5,random_state=42,shuffle=True)

accuracies = []

y = classes

for iteration, data in enumerate(kf.split(X), start=1):

    data_train   = X[data[0]]
    target_train = y[data[0]]

    data_test    = X[data[1]]
    target_test  = y[data[1]]

    clf = MLPClassifier()
    clf.fit(data_train,target_train)

    preds = clf.predict(data_test)

    # for the current fold only    
    accuracy = accuracy_score(target_test,preds)

    print("Training ke-", iteration)
    print(classification_report(target_test,preds))
    print("======================================================")

    accuracies.append(accuracy)

# this is the average accuracy over all folds
average_accuracy = np.mean(accuracies)


print("Rata-rata Accuracy: ", average_accuracy)

Training ke- 1
              precision    recall  f1-score   support

    negative       0.77      0.79      0.78       680
     neutral       0.78      0.61      0.68       239
    positive       0.87      0.90      0.89      1281

    accuracy                           0.83      2200
   macro avg       0.81      0.77      0.78      2200
weighted avg       0.83      0.83      0.83      2200

Training ke- 2
              precision    recall  f1-score   support

    negative       0.80      0.78      0.79       706
     neutral       0.74      0.70      0.71       220
    positive       0.89      0.91      0.90      1274

    accuracy                           0.85      2200
   macro avg       0.81      0.79      0.80      2200
weighted avg       0.84      0.85      0.84      2200

Training ke- 3
              precision    recall  f1-score   support

    negative       0.80      0.80      0.80       682
     neutral       0.86      0.72      0.78       215
    positive       0.89      0

In [51]:
#Prediction
original_text =  '''
Abid sangat suka
'''

# Feature Extraction
text = count_vect.transform([process_text(original_text)])

# Kita prediksi sentimennya
result = model.predict(text)[0]
print("Sentiment:")
print()
print(result)

Sentiment:

positive


In [53]:
def get_centiment_nn(original_text):
    text = count_vect.transform([process_text(original_text)])
    result = model.predict(text)[0]
    return result

print('Saya suka makan nasi')
print(get_centiment_nn("Saya suka makan nasi"))

Saya suka makan nasi
positive
